In [1]:
import json
import os
import ast
import pandas as pd
rest_reviews=pd.read_csv('restaurant_reviews_newcols.csv',nrows=1000)

In [42]:
emphasis_text=[x for x in rest_reviews['text'] if isinstance(x,str)]
emphasis_count=[x.count('!') for x in rest_reviews['text'] if isinstance(x,str)]
temp=pd.DataFrame()
temp['text']=emphasis_text
temp['count']=emphasis_count

In [54]:
check=temp[temp['count']>50]['text']

In [55]:
print(check)

23807      So I'm a loyal Hooters customer !! I've been t...
46946      "Wy Knot Cafe" formally known as "Perc Up" is ...
48509      I've had good and bad experiences with HOME DE...
117821     Terrible service from Jay! We went here for my...
121465     If I could give it a negative star rating I wo...
123420     this review is loooooong overdue for La Dolce ...
174862     The most wonderful store .. the most hap, happ...
183716     Food took a long time and horrible I would not...
185715     I totally agree with Mr. or Mrs. Jen K. !!!!!!...
197754     Black Friday was a absolute mess and this Walm...
225378     I ordered a breakfast burrito egg, bacon ,pota...
225379     this place sucks!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...
225816     Today at Dom Demarcos, Las Vegas, NV was the W...
237835     WE MOTHERF'IN' BALLLLLLLLLLLLLLLLLLL'  WE DON'...
251523     You can have the best food but the worst servi...
259244     HORRIBLE EXPERIENCE!!! STAY AWAY!!!!!!!!!!!!!!...
292182     I don't ventu

In [66]:
def category_filter(df):
    import re
    global search_category
    searched=search_category.split(' ')
    
    if isinstance(df['text'],str):
        temp=df['text'].split(', ')
        checksum=sum([searched.count(x) for x in temp])
        return checksum
    else:
        return 0

What are you searching for today?Lunch with boss


In [78]:
search_category=input('What are you searching for today?')
rest_reviews['flag']=rest_reviews.apply(category_filter,axis=1)
[print(i) for i in rest_reviews[rest_reviews['flag']>0]]

What are you searching for today?formal boss
Unnamed: 0
review_id
user_id
business_id
stars
date
text
useful
funny
cool
user_rating_count
average_rating_rest
rest_review_count
flag


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [3]:
#Trying to use LDA to model topics for reviews
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
import pprint

In [7]:
documents=rest_reviews['text']
stoplist = list(STOPWORDS)
texts = [[word for word in document.lower().split() if (word not in stoplist) & (word.isalnum())]
         for document in documents if isinstance(document,str)]
texts

[['pizza',
  'best',
  'prefer',
  'flamingo',
  'fort',
  'chef',
  'better',
  'ny',
  'style',
  'pizzeria',
  'cosmo',
  'priced',
  'quality',
  'lack',
  'personality',
  'better',
  'pick',
  'youre',
  'going',
  'italian',
  'family',
  'home',
  'people',
  'actually',
  'care',
  'like',
  'dont',
  'pizzeria',
  'dont',
  'care'],
 ['love',
  'fiance',
  'atleast',
  'portions',
  'food',
  'love',
  'carne',
  'great',
  'lunch',
  'leticia',
  'super',
  'nice',
  'cares',
  'think',
  'try',
  'cheese',
  'enchiladas',
  'sauce',
  'different'],
 ['dry',
  'corn',
  'rib',
  'tips',
  'fat',
  'mushy',
  'want',
  'bbq',
  'neighborhood',
  'john',
  'mulls',
  'roadkill',
  'trust'],
 ['place',
  'favorite',
  'thai',
  'place',
  'alllll',
  'started',
  'known',
  'got',
  'service',
  'started',
  'suck',
  'portion',
  'sizes',
  'got',
  'cut',
  'huge',
  'problem',
  'paying',
  'way',
  'time',
  'went',
  'pork',
  'pad',
  'se',
  'ew',
  'tasted',
  'finished

In [36]:
#In the above code, we tokenize all of the reviews and remove stop words
#remove words that appear only once 
#frequency of the token across ALL review

#counting the frequency of token across ALL reviews
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
#removing tokens that occur only once across all reviews
texts = [[token for token in text if frequency[token] > 1 & frequency[token] <10000] for text in texts]
print(*texts)

['pizza', 'best', 'prefer', 'chef', 'better', 'style', 'priced', 'quality', 'lack', 'better', 'pick', 'going', 'italian', 'family', 'home', 'people', 'actually', 'care', 'like', 'dont', 'dont', 'care'] ['love', 'fiance', 'portions', 'food', 'love', 'carne', 'great', 'lunch', 'super', 'nice', 'think', 'try', 'cheese', 'sauce', 'different'] ['dry', 'corn', 'rib', 'fat', 'want', 'bbq', 'neighborhood'] ['place', 'favorite', 'thai', 'place', 'started', 'known', 'got', 'service', 'started', 'portion', 'got', 'cut', 'huge', 'problem', 'paying', 'way', 'time', 'went', 'pork', 'pad', 'tasted', 'finished', 'plate', 'know', 'manager', 'nice', 'pad', 'thai', 'pad', 'know', 'knows', 'asked', 'new', 'portion', 'great', 'food', 'worth', 'year', 'pay', 'dinner', 'hungry', 'sorry'] ['delicious', 'healthy', 'steak', 'fish', 'pork', 'awesome', 'service', 'bad', 'thing', 'worth'] ['place', 'customer', 'service', 'dont', 'serve', 'food', 'order', 'pizza', 'dont', 'times', 'gone', 'seen', 'bartenders', 'got

In [37]:
dictionary = corpora.Dictionary(texts) #word id-word pairs
corpus = [dictionary.doc2bow(text) for text in texts] #word id-frequence of word pairs

In [44]:
import collections as c
emptylist=[]
[emptylist.extend(x) for x in texts]
c.Counter(emptylist).most_common()


[('food', 489),
 ('place', 380),
 ('great', 357),
 ('good', 333),
 ('like', 318),
 ('service', 239),
 ('love', 186),
 ('time', 170),
 ('best', 152),
 ('got', 133),
 ('chicken', 133),
 ('nice', 130),
 ('order', 128),
 ('try', 123),
 ('little', 118),
 ('come', 114),
 ('definitely', 110),
 ('came', 108),
 ('went', 105),
 ('restaurant', 105),
 ('people', 101),
 ('friendly', 100),
 ('eat', 97),
 ('ordered', 97),
 ('fresh', 97),
 ('going', 92),
 ('pretty', 92),
 ('staff', 90),
 ('want', 89),
 ('recommend', 89),
 ('lunch', 88),
 ('know', 85),
 ('delicious', 85),
 ('menu', 85),
 ('better', 84),
 ('sure', 77),
 ('bar', 76),
 ('fried', 75),
 ('way', 73),
 ('wait', 73),
 ('2', 72),
 ('thai', 71),
 ('tried', 71),
 ('server', 71),
 ('favorite', 69),
 ('salad', 69),
 ('think', 68),
 ('prices', 68),
 ('pizza', 67),
 ('atmosphere', 67),
 ('table', 67),
 ('cheese', 66),
 ('worth', 66),
 ('right', 66),
 ('night', 65),
 ('dinner', 64),
 ('small', 64),
 ('sushi', 63),
 ('excellent', 63),
 ('hot', 60),
 ('

In [38]:
#Set parameters
num_topics = 100
passes = 50
lda = LdaModel(corpus,
              id2word=dictionary,
              num_topics=num_topics,
              passes=passes)

/Users/sanjanarosario/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [39]:
lda.print_topics()

[(18,
  '0.045*"time" + 0.042*"room" + 0.026*"clean" + 0.024*"stay" + 0.021*"food" + 0.021*"decided" + 0.021*"minutes" + 0.020*"disappointed" + 0.017*"good" + 0.017*"got"'),
 (77,
  '0.035*"food" + 0.025*"second" + 0.025*"wines" + 0.025*"tour" + 0.024*"wine" + 0.023*"time" + 0.017*"like" + 0.017*"place" + 0.017*"way" + 0.017*"bus"'),
 (17,
  '0.046*"going" + 0.035*"experience" + 0.026*"recommend" + 0.026*"kind" + 0.025*"bar" + 0.021*"wine" + 0.019*"bottle" + 0.018*"white" + 0.018*"great" + 0.017*"good"'),
 (88,
  '0.033*"food" + 0.018*"people" + 0.017*"place" + 0.017*"little" + 0.017*"coffee" + 0.017*"piece" + 0.017*"dessert" + 0.016*"bad" + 0.016*"happy" + 0.016*"restaurant"'),
 (75,
  '0.037*"food" + 0.029*"like" + 0.022*"pho" + 0.022*"lot" + 0.015*"come" + 0.015*"great" + 0.015*"menu" + 0.015*"buffet" + 0.015*"vegas" + 0.015*"decent"'),
 (66,
  '0.042*"thai" + 0.024*"place" + 0.024*"good" + 0.023*"ate" + 0.016*"plenty" + 0.015*"little" + 0.013*"chicken" + 0.013*"fresh" + 0.013*"fala